In [2]:
import CaboCha

c = CaboCha.Parser()

txt = input()

tree =  c.parse(txt)
cabocha_lattice = tree.toString(CaboCha.FORMAT_LATTICE)

ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。


In [3]:
class Morph:
    def __init__(self,surface,attr):
        self.surface = surface
        self.base = attr[6]
        self.pos = attr[0]
        self.pos1 = attr[1]

In [4]:
class Chunk:
    def __init__(self,morphs,dst,srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

In [5]:
from collections import defaultdict

res = []
for s in cabocha_lattice.split("EOS\n"):
    sent = []
    src_d = defaultdict(list)

    for line_ind, line in enumerate(s.split("\n")):

        if line == "":
            continue

        elif line[0] == "*":
            morphs = []

            _, ind, dst, *_ = line.split()
            dst = int(dst.replace("D", ""))
            src_d[dst].append(int(ind))

            if ind != 0:
                srcs = src_d[int(ind)]
                chunk = Chunk(morphs, dst, srcs)
                sent.append(chunk)
        else:
            surface, attr = line.split()
            attr = attr.split(",")
            morphs.append(Morph(surface, attr))

    res.append(sent)

In [7]:
from itertools import combinations
for sent in [res[0]]:
    noun_index = []
    for ind,chunk in enumerate(sent):
        if "名詞" in set([morph.pos for morph in chunk.morphs]):
            noun_index.append(ind)
            
    for i,j in combinations(noun_index,2):
        X_pass = [i]
        Y_pass = [j]
        
        ind = sent[i].dst
        while True:
            if ind == -1:
                break
            else:
                X_pass.append(ind)
                ind = sent[ind].dst
        
        ind = sent[j].dst
        while True:
            if ind == -1:
                break
            else:
                Y_pass.append(ind)
                ind = sent[ind].dst
                
        X_chunk = []
        for morph in sent[i].morphs:
            if morph.pos == "名詞":
                X_chunk.append("X")
            elif morph.pos == "記号":
                continue
            else:
                X_chunk.append(morph.surface)
        
        
        Y_chunk = []
        for morph in sent[j].morphs:
            if morph.pos == "名詞":
                Y_chunk.append("Y")
            elif morph.pos == "記号":
                continue
            else:
                Y_chunk.append(morph.surface)
        
        disp = "".join(X_chunk)
        if j in X_pass:
            for xind in X_pass[1:]:
                if xind == j:
                    break
                disp += ("->" + "".join([morph.surface for morph in sent[xind].morphs if morph.pos != "記号"]))
            disp += ("->" + "".join(Y_chunk))
            
        else:
            for xind in X_pass[1:-1]:
                disp += ("->" + "".join([morph.surface for morph in sent[xind].morphs if morph.pos != "記号"]))
            disp += (" | " + "".join(Y_chunk))
            for yind in Y_pass[1:-1]:
                disp += ("->" + "".join([morph.surface for morph in sent[yind].morphs if morph.pos != "記号"]))
            
            disp += (" | " + "".join([morph.surface for morph in sent[X_pass[-1]].morphs if morph.pos != "記号"]))
            
        print(disp)

XXは | Yに関する->最初の->会議で | 作り出した
XXは | Yの->会議で | 作り出した
XXは | Yで | 作り出した
XXは | YYという->用語を | 作り出した
XXは | Yを | 作り出した
Xに関する->Yの
Xに関する->最初の->Yで
Xに関する->最初の->会議で | YYという->用語を | 作り出した
Xに関する->最初の->会議で | Yを | 作り出した
Xの->Yで
Xの->会議で | YYという->用語を | 作り出した
Xの->会議で | Yを | 作り出した
Xで | YYという->用語を | 作り出した
Xで | Yを | 作り出した
XXという->Yを
